In [1]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import itertools as iter
from random import shuffle

physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))


DATA_PATH = os.getenv('DATA_PATH')
print(DATA_PATH)


2023-09-06 22:41:35.147933: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 22:41:35.591196: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1
/mnt/e/ML/MonetGAN


2023-09-06 22:41:36.502486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-06 22:41:36.520729: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-06 22:41:36.521040: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
T=1000
beta = np.linspace(1e-4, 0.04, T)
alpha = 1-beta
alphaBar = [np.prod(alpha[0:i+1]) for i,el in enumerate(alpha)]
IMAGE_SIZE = [256,256]
tValues = np.arange(0,T)
# alphaBar = np.cos((tValues/(T+1) + 0.008)/(1 + 0.008) * np.pi/2) 
# np.random.shuffle(tValues)


#Cosine schedule
# def f(t):
#     return np.cos((t/T + 0.008)/(1 + 0.008) * np.pi/2)**2

# alphaBar = f(tValues)
# alpha = (alphaBar[1:])/(alphaBar[0:-1])
# alpha = np.insert(alpha, 0, alphaBar[0])
# beta = 1 - alpha


# Simple linear
alphaBar = np.clip(1-tValues/T, 0.0001, 0.9999)
alpha = (alphaBar[1:])/(alphaBar[0:-1])
alpha = np.insert(alpha, 0, alphaBar[0])
beta = 1 - alpha



def forwardStep(x0Image,t):
    noise = np.random.normal(0, 1, x0Image.shape)
    noisy = np.sqrt(alphaBar[t])*x0Image + np.sqrt(1-alphaBar[t])*noise
    return noise, noisy


In [3]:
variance = np.append(np.array([0]), beta[1:]*(1-alphaBar[0:T-1])/(1-alphaBar[1:T]))
def backwardStep(xt, t, predNoise):
    if t>1:
        randNoise = tf.random.normal(xt.shape)
    else:
        randNoise= tf.zeros(xt.shape)

    # sigPred = np.sqrt(beta)
    # xtPrev = (1/np.sqrt(alpha[t])) * (xt - (1-alpha[t])/(np.sqrt(1-alphaBar[t])) * predNoise) + sigPred[t]*randNoise

    x0Pred = (1/np.sqrt(alphaBar[t])) * (xt - np.sqrt(1-alphaBar[t]) * predNoise)
    x0Pred = np.clip(x0Pred, -1.0, 1.0)
    meanPred = beta[t] * (np.sqrt(alphaBar[t-1]))/(1-alphaBar[t]) * x0Pred + (1-alphaBar[t-1])*(np.sqrt(alpha[t]))/(1-alphaBar[t]) * xt
    xtPrev = meanPred + np.sqrt(variance[t]) * randNoise

    return xtPrev



In [4]:
def postprocessImage(im):
    return np.clip(im/2+0.5,0,1)

## Tensorflow

In [5]:
model = tf.keras.models.load_model(os.path.join(DATA_PATH, "unet_T1000_K16_DIM128epoch_822"))
IMAGE_SIZE=[128,128]

2023-09-06 22:41:42.498883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-06 22:41:42.499234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-06 22:41:42.499538: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-06 22:41:42.960647: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-06 22:41:42.961019: I tensorflow/compile

In [6]:
def generateImages(BatchSize=64):
    initInput = tf.random.normal((BatchSize, *IMAGE_SIZE, 3), dtype=tf.float32)
    samples = initInput

    for t in np.arange(1,T)[::-1]:
        predNoise = model((samples, tf.repeat(t, repeats=BatchSize)), training=False)
        xt = samples
        
        new = backwardStep(xt, t, predNoise=predNoise)

        samples = new
        
    return samples



In [7]:
N=2

for n in tqdm(range(N)):
    images = generateImages(100)
    images = postprocessImage(images)
    for i in range(images.shape[0]):
        im = Image.fromarray(np.array(images[i,:,:,:]*255).astype(np.uint8), "RGB" )
        im.save(os.path.join(DATA_PATH,"results/"+ str(n*100+i)+".jpg"),quality=80)
        

 50%|█████     | 1/2 [10:08<10:08, 608.08s/it]


KeyboardInterrupt: 

In [ ]:
im = Image.fromarray(np.array(images[0,:,:,:]*255).astype(np.uint8), "RGB" )
# np.array(img).astype(np.uint8).transpose(1,2,0)

In [ ]:
os.path.join(DATA_PATH, "results", str(i)+".jpg")

In [ ]:
im.save(os.path.join(DATA_PATH,"results/"+ str(i)+".jpg"))